In [1]:
import numpy as np
import pandas as pd

In [2]:
def compare(fitness_a, fitness_b):
    # if a dominates b, return will be 1, -1 if the contrary
    # if there is no dominance, 0 will be returned
    try:
        m = (fitness_a[0] - fitness_b[0]) / (fitness_a[1] - fitness_b[1])
    except ZeroDivisionError:
        return 0
    if m > 0:
        if fitness_a[0] < fitness_b[0]:
            return 1
        return -1
    return 0


In [3]:
def get_parents(parent_F:list, parent_G:list=None, parent_idx:list=None):
	levels  = np.array(parent_F)[:, 2]
	max_idx = np.argmin(levels)

	if parent_G is not None:
		return parent_G[max_idx]
	if parent_idx is not None:
		return parent_idx[max_idx]
	return max_idx

In [4]:
def mix_parents(parent_a:list, parent_b:list, ratio:float = 0.5, get_indexes:bool=False):
	genotype_len = len(parent_a)

	a_idxs = set()
	acum = 0
	for i in range(genotype_len):
		acum += ratio
		if acum >= 1:
			acum -= 1
			a_idxs.add(i)
	b_idxs = {i for i in range(genotype_len)} - a_idxs

	a_idxs = list(a_idxs)
	b_idxs = list(b_idxs)
	child  = np.zeros((genotype_len), bool)

	child[a_idxs] = parent_a[a_idxs].copy()
	child[b_idxs] = parent_b[b_idxs].copy()
	if get_indexes:
		return child, (a_idxs, b_idxs)
	return child

In [5]:
def NSGA2(fitness):
    n = len(fitness)
    levels = np.zeros(n)
    dominates = [[] for _ in range(n)]
    is_dominated_by = {}
    for i in range(n):
        is_dominated_by[i] = 0
    for idx_first_point, first_point in enumerate(fitness):
        for idx_second_point in range(idx_first_point+1, len(fitness)):
            second_point = fitness[idx_second_point]
            m = compare(first_point, second_point)
            if m == 1: #first_point dominates
                is_dominated_by[idx_second_point] = is_dominated_by.get(idx_second_point, 0) + 1
                dominates[idx_first_point].append(idx_second_point)
            elif m == -1:
                is_dominated_by[idx_first_point] = is_dominated_by.get(idx_first_point, 0) + 1
                dominates[idx_second_point].append(idx_first_point)
        
    #sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1]))

    curr_level = 0
    while len(is_dominated_by) > 0:
  
        non_dominated_indices = [idx for idx, dominance_count in is_dominated_by.items() if dominance_count == 0]
        for idx in non_dominated_indices:
            
            for dominated_idx in dominates[idx]:
                is_dominated_by[dominated_idx] -= 1
                
            levels[idx] = curr_level
            del is_dominated_by[idx]
        curr_level+=1
        
    return levels
    

In [6]:
def get_fitness:
    

SyntaxError: invalid syntax (2632364954.py, line 1)

# Initial population

In [ ]:
def solve(n, pCrossover, pMutation):
    #read csv
    filename = 'wine.csv'  
    df = pd.read_csv(filename)
    num_features = df.shape[1] -1
    
    #generate p0
    p0 = np.random.randint(2, size=(n, num_features))
    print(p0)
    print(p0[1][0],p0[1][1])


In [63]:
class mock_solver:
	def __init__(self):
		self.mutations = 0
		self.crosses   = 0
		self.updates   = 0

	def generate_initial_population(self, p_size) -> list:
		return [None for i in range(p_size)]

	def compare(self, fitness_a, fitness_b):
		...

	def get_parents(self, parent_F:list, parent_G:list=None, parent_idx:list=None):
		return 0

	def mix_parents(self, parent_a, parent_b, ratio:float=0.5):
		self.crosses   += 1
	def mutate(self, g):
		self.mutations += 1

	def get_fitness(self, G:list, F:list=None):
		if F is None:
			return [0 for i in range(len(G))]

	def get_elite(self, G:list, F:list) -> (list, list):
		return ([], [])

	def solved(self):
		return False

	def deep_copy(self, G, population:bool=True):
		if population:
			self.updates += 1
			return [None for i in enumerate(G)]
		return None

	def has_progress_metric(self):
		return False
	def get_progress_metric(self, F:list):
		...
	def get_progress_max(self, ):
		...

	def __str__(self):
		mut_ratio   = self.mutations / self.updates
		cross_ratio = self.crosses   / self.updates

		ret_string  = f'Mock test of {self.updates} iterations'
		ret_string += f'has a {mut_ratio:.2f} mutation and {cross_ratio:.2f} cross ratio'

		return ret_string

In [65]:
def solve_genetic(
	n
	,solver
	,max_it:int       = 1000
	,pCross:float     = 0.5
	,pMut:float       = 0.5
	,tournament_n:int = 3
	,rng_gen=np.random.default_rng
	# Formatting options for progress bar
	,MAX_F_LEN:int = 30
	,MAX_K_LEN:int = 20
	,SPINNER = ['\\', '|', '/', '-']
):
	rng = rng_gen()

	SPINNER_LEN = len(SPINNER)

	G  = solver.generate_initial_population(n)
	Gi = [None for i in range(n)]
	F  = solver.get_fitness(G)

	elite, Fe = solver.get_elite(G, F)

	for k in range(1, max_it+1):
		if solver.solved(): break

		for i, gntype in enumerate(G):
			if rng.random() < pCross:
				parent_candidates_idx = rng.integers(low=0, high=n, size=tournament_n)
				parent_idx = solver.get_parents(
					[F[idx] for idx in parent_candidates_idx]
					,parent_idx=parent_candidates_idx
				)
				Gi[i] = solver.mix_parents(gntype, G[parent_idx])
			else:
				Gi[i] = solver.deep_copy(gntype, population=False)

			if rng.random() < pMut:
				Gi[i] = solver.mutate(gntype)
		#-----------
		
		Fi = solver.get_fitness(Gi)

		elite_i, Fe_i = solver.get_elite(Gi, Fi)
		elite, Fe     = solver.get_elite(elite + elite_i,Fe + Fe_i)

		G = solver.deep_copy(Gi)
		F = solver.get_fitness(G)

		if k%10 == 1 or k==max_it:
			spinner_idx:int = int(k/10)%SPINNER_LEN
			k_percentage    = k/float(max_it)*100
			filled_k = int(MAX_K_LEN * k  // max_it)

			k_bar = '#'*filled_k + '-'*(MAX_K_LEN - filled_k)
			
			print(f'\r{SPINNER[spinner_idx]}', end='')
			print(f' Iteration:[{k_bar}]{max_it}:{k_percentage:.1f}%', end='\r')
			if solver.has_progress_metric():
				
	#-------------
	print('\nFinished!')
	return elite

solver = mock_solver()

solve_genetic(50, solver)
print(solver)

\ Iteration:[####################]1000:100.0%
Finished!
Mock test of 1000 iterationshas a 25.07 mutation and 25.04 cross ratio


# test functions

In [ ]:
solve(10,0.5,0.5)

1


In [ ]:
def test_NSGA():
    fitn = [
        [1,2],
        [5,3],
        [6,5],
        [4,2],
        [2,4],
        [3.88,4.14],
        [3,3],
        [2,1]    
    ]

    levels = NSGA2(fitn)
    print(levels)